In [49]:
import tkinter as tk
import cv2
from time import sleep
import numpy as np
from tkinter import filedialog

def vehicle_count():
    # Initialize variables
    count_in = 0
    count_out = 0
    line_pos = 550
    offset = 6
    detect = []
    # Prompt the user to select the video file
    video_file_path = filedialog.askopenfilename()

    # Check if the user has selected a file
    if video_file_path:
        # Open the video file
        cap = cv2.VideoCapture(video_file_path)
        subtract = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

        while True:
            # Read a frame from the video
            ret, frame = cap.read()

            # Apply background subtraction to the frame
            mask = subtract.apply(frame)
            _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw a line on the frame
            cv2.line(frame, (25, line_pos), (1200, line_pos), (255, 255, 0), 3)

            # Loop over the contours
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)       
                area = cv2.contourArea(cnt)

                # Check if the contour is large enough and has the dimensions of a car or truck/bus
                if area > 100:
                    if w >= 80 and h >= 80:
                        # Get the center of the contour
                        centre = (x + w//2, y + h//2)

                        # Classify the vehicle as car or truck/bus based on its aspect ratio
                        aspect_ratio = w / float(h)
                        if aspect_ratio > 1.5:
                            # Truck/bus
                            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 3)
        
                        else:
                            # Car
                            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
                         

                        # Add the center to the list of detected vehicles
                        detect.append(centre)

                        # Draw a circle around the center
                        cv2.circle(frame, centre, 4, (0, 0, 255), -1)

                        # Check if the vehicle has crossed the line
                        for (x, y) in detect:
                            if y < (line_pos + offset) and y > (line_pos - offset):
                                # Vehicle is crossing the line
                                if x < 500:
                                    # Vehicle is going out of the line
                                    count_out += 1
                                    cv2.line(frame, (50, line_pos), (1200, line_pos), (0, 0, 255), 3)
                                    detect.remove((x, y))

                                else:
                                    # Vehicle is coming towards the line
                                    count_in += 1
                                    cv2.line(frame, (50, line_pos), (1200, line_pos), (255, 0, 255), 3)
                                    detect.remove((x, y))


            cv2.putText(frame, "Press 'q' for Exit", (100, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255), 2)                        
            # Display the vehicle counts
            cv2.putText(frame, "Vehicle-In Count: " + str(count_in), (100, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 255), 2)
            cv2.putText(frame, "Vehicle-Out Count: " + str(count_out), (100, 150), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Output", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

        print("Vehicle-Out Detected:", str(count_out))
        print("Vehicle-In Detected:", str(count_in))

# Create a window
root = tk.Tk()
# Set the window title and size
root.title("Car Detection and Counting System")
root.geometry("800x600")
root.state('zoomed')



# Create a label at the top with bigger font
label = tk.Label(root, text="Car Detection and Counting System", font=("Arial", 50), fg="gray")
label.pack(pady=50)

# Create a button that calls the vehicle_count function
button = tk.Button(root, text="Upload Video", command=vehicle_count, bg="#4CAF50", fg="white", font=("Arial", 24))
button.pack(pady=100)


# Run the window
root.mainloop()

Vehicle-Out Detected: 5
Vehicle-In Detected: 1
